In [18]:
#%
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import tqdm
import cx_Oracle

connection = cx_Oracle.connect(user='scott', password='tiger', dsn='192.168.119.119:1521/dink11.dbsvr')
cursor = connection.cursor()
sql="select categorySmall, convenienceToEmbedding from CATEGORYembedding"
cursor.execute(sql)
rows = cursor.fetchall()
columns = ['categorySmall','convenienceToEmbedding']
category = pd.DataFrame(rows, columns=columns)

sql="SELECT c.email as email,ch.categorySmall, ch.amount FROM cardhistory ch JOIN card c ON ch.cardid = c.cardid WHERE ch.cardhisdate >= TO_DATE('2023-07-07', 'YYYY-MM-DD')"
cursor.execute(sql)
rows = cursor.fetchall()
columns = ['email','categorySmall','amount']
data = pd.DataFrame(rows, columns=columns)
data['categoryEmbedding'] = data['categorySmall'].replace(category.set_index('categorySmall')['convenienceToEmbedding'])
userList=data['email'].unique()

In [19]:
mode_card_df = {'email': [], 'categorySmall' : [], 'embedding_mode' : [],'amount':[]}
for i, userId in tqdm.tqdm(enumerate(userList)):
    '''
    _df : 카드 거래 내역 중 카드번호 같은 것 끼리 모은 임시 df
    mean_card_df : 카드번호 같은 것들 중, 최빈값 업종을 뽑아서 대표 업종으로 만든 df
    '''
    _df = data[data['email']==userId][['categorySmall', 'categoryEmbedding','amount']]
    # print(_df)
    # print(_df['categorySmall'].mode().iloc[0])
    # print(_df['amount'].mean())

    mode_card_df['email'].append(userId)
    mode_card_df['categorySmall'].append(_df['categorySmall'].mode().iloc[0])
    mode_card_df['embedding_mode'].append(_df['categoryEmbedding'].mode().iloc[0])
    mode_card_df['amount'].append(_df['amount'].mean())
    
# mode_card_df = pd.DataFrame(mode_card_df)
# print(mode_card_df)
print(len(mode_card_df['email']))
print(len(mode_card_df['categorySmall']))
print(len(mode_card_df['amount']))
mode_card_df = pd.DataFrame(mode_card_df)
mode_card_df['amount'] = mode_card_df['amount'].astype(int)
mode_card_df

7343it [28:21,  4.32it/s]

7343
7343
7343


,email,categorySmall,embedding_mode,amount
0,pooh5045@naver.com,한식,0.178922,25612
1,icoe@gmail.com,제과점,0.432291,29808
2,gyeongsuson@naver.com,제과점,0.432291,63510
3,zjang@dreamwiz.com,제과점,0.178922,128854
4,ohaeun@hanmail.net,편의점,1.000000,112158
...,...,...,...,...
7338,hasunja@hotmail.com,한식,0.178922,107880
7339,minsu61@daum.net,한식,0.178922,78879
7340,songyeongsug@daum.net,한식,0.178922,47522
7341,mijeong17@live.com,한식,0.178922,115254


In [20]:
from sklearn.cluster import KMeans

n_clusters = 5
X = np.array(list(zip(mode_card_df['embedding_mode'].values, mode_card_df['amount'].values)))
print(X.shape)

kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
cluster_labels = kmeans.labels_
print(len(cluster_labels))
mode_card_df['Cluster'] = cluster_labels
mode_card_df['Cluster']=mode_card_df['Cluster'].astype(str)

(7343, 2)


7343


In [21]:
mode_card_df['Cluster'].value_counts()

4    2453
1    1810
0    1663
3    1014
2     403
Name: Cluster, dtype: int64

In [22]:
connection = cx_Oracle.connect(user='scott', password='tiger', dsn='192.168.119.119:1521/dink11.dbsvr')
cursor = connection.cursor()
insertSql="INSERT INTO clusterTable (email,clusterNum) values(:1,:2)"
for i in range(len(mode_card_df)):
    cursor.execute(insertSql, (mode_card_df.loc[i,'email'],mode_card_df.loc[i,'Cluster']))
connection.commit()
cursor.close()
connection.close()  